In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
from os import listdir
from os.path import  isfile
import re
from collections import defaultdict

MAX_DOC_LENGTH=500
unknown_ID=-999
padding_ID=-1000

def gen_data_and_vocab():
    def collect_data_from(parent_path, newsgroup_list, word_count=None):
        data=[]
        for group_id, newsgroup in enumerate(newsgroup_list):
            dir_path= parent_path + '/' + newsgroup+'/'
            files= [(filename, dir_path+ filename)
                    for filename in listdir(dir_path)
                    if isfile(dir_path+ filename)]
            files.sort()
            label= group_id
            print('processing: {} - {}'.format(group_id, newsgroup))

            for filename, file_path in files:
                with open(file_path,'rb') as f:
                    text= f.read().decode('UTF-8', errors='ignore').lower()
                    words= re.split('\W+', text)
                    if word_count is not None: # chi ap dung cho train data
                        for word in words:
                            word_count[word]+=1
                    content= ' '.join(words)
                    assert  len(content.splitlines())==1
                    data.append(str(label)+ '<fff>'
                                + filename+ '<fff>'+ content)
        return data

    word_count= defaultdict(int)

    path= '/content/drive/My Drive/Python code/DS_Lab2/Data/'
    parts=[path + dir_name + '/' for dir_name in  listdir(path)
           if not isfile(path + dir_name)]

    train_path, test_path= (parts[0], parts[1]) if 'train' in parts[0]\
        else (parts[1], parts[0])

    newsgroups_list= [newsgroup  for newsgroup in listdir(train_path)]
    newsgroups_list.sort()

    #xay dung tu dien va thu thap du lieu
    train_data= collect_data_from(
        parent_path= train_path,
        newsgroup_list= newsgroups_list,
        word_count= word_count
    )
    vocab=[word for word, freq in
           zip(word_count.keys(), word_count.values())
           if freq > 10]
    vocab.sort()
    with open('/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/vocab-raw.txt','w') as f:
        f.write('\n'.join(vocab))

    test_data = collect_data_from(
        parent_path=test_path,
        newsgroup_list=newsgroups_list
    )

    with open('/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-train-raw.txt','w') as f:
         f.write('\n'.join(train_data))
    with open('/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-test-raw.txt','w') as f:
         f.write('\n'.join(test_data))


def encode_data(data_path, vocab_path):
    with open(vocab_path) as f:
        vocab= dict([(word, word_ID+2)
                     for word_ID, word in enumerate(f.read().splitlines())])
    with open(data_path) as f:
        documents= [(line.split('<fff>')[0],
                     line.split('<fff>')[1],
                     line.split('<fff>')[2])
                    for line in f.read().splitlines()]
    
    encoded_data= []
    for docs in documents:
        label, doc_id, text= docs
        words= text.split()[:MAX_DOC_LENGTH]
        sentence_length= len(words)
        
        encode_text= []
        for word in words:
            if word in vocab:
                encode_text.append(str(vocab[word]))
            else:
                encode_text.append((str(unknown_ID)))
        if len(words) <MAX_DOC_LENGTH:
            num_padding= MAX_DOC_LENGTH - len(words)
            for i in range(num_padding):
                encode_text.append(str(padding_ID))
                
        encoded_data.append(str(label)+ '<fff>' +
                           str(doc_id) +'<fff>'+
                           str(sentence_length)+ '<fff>'+
                           ' '.join(encode_text))

    dir_name= '/'.join(data_path.split('/')[:-1])
    file_name= '-'.join(data_path.split('/')[-1].split('-')[:-1]) + '-encoded.txt'
    with open(dir_name +'/'+ file_name, 'w') as f:
        f.write('\n'.join(encoded_data))

if __name__== "__main__":
    gen_data_and_vocab()
    train_data_path='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-train-raw.txt'
    test_data_path='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-test-raw.txt'
    vocab_path='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/vocab-raw.txt'
    # encode_data(train_data_path, vocab_path)
    encode_data(test_data_path, vocab_path)

In [0]:
with open('/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/chuc2.txt') as f:
    a= f.read()
    a=a.replace("-999","0")
    a=a.replace("-1000","1")

with open('/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/chuc2.txt','w') as f:
    f.write(a)

In [0]:
file='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-test-encoded.txt'
with open(file) as f:
    a= f.read()
    a=a.replace("-999","0")
    a=a.replace("-1000","1")

with open(file,'w') as f:
    f.write(a)

In [0]:
file='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-train-encoded.txt'
with open(file) as f:
    a= f.read()
    a=a.replace("-999","0")
    a=a.replace("-1000","1")

with open(file,'w') as f:
    f.write(a)

In [0]:
# import tensorflow.compat.v1 as tf2
import numpy as np
import random 


class DataReader:
    def __init__(self, data_path, batch_size, vocab_size):
        self._batch_size= batch_size
        with open(data_path) as f:
            d_lines= f.read().splitlines()
        
        self._data=[]
        self._labels=[]
        self._sentence_lengths=[]
        for data_id, line in enumerate(d_lines):
            # tfidf_vec= [0.0 for i in range(vocab_size)]
            if len(line) > 1:
                feature= line.split('<fff>')
                # print("*"+feature[2] +"*")
                # print("*"+feature[1] +"*")
                # print("*"+feature[0] +"*")
                label, doc_id, sentence_length= int(feature[0]), int(feature[1]), int(feature[2])
                # tokens= feature[3].split()
                tokens= feature[3].split()
                vector= [int(token) for token in tokens]
                # for token in tokens:
                    # print("tên file:" ,data_path)
                    # print("dòng thứ: ", data_id)
                    # print("tokens:", tokens[:10])
                    # print(token)
                    # ind, value= int(token.split(":")[0]), \
                    #             float(token.split(":")[1])
                    # tfidf_vec[ind]= value
                    
                
                # self._data.append(tfidf_vec)
                self._data.append(vector)
                self._labels.append(label)
                self._sentence_lengths.append(sentence_length)

        self._data= np.array(self._data)
        self._labels= np.array(self._labels)
        self._sentence_lengths= np.array(self._sentence_lengths)

        self._num_epoch=0
        self._batch_id=0

    def next_batch(self):
        #lấy data từ trong batch tiếp theo và xáo chộn lên
        start= self._batch_id * self._batch_size
        end= start + self._batch_size
        self._batch_id +=1

        if end + self._batch_size > len(self._data):
        # khi đến batch cuối, một epoch có kích thước bằng data, nên 
        # khi chọn data set cho từng patch cần xáo chộn ngẫu nhiên rồi chọn cho đủ patch
        # dù cho trong 1 epoch có thể không chứa toàn bộ dữ liệu nhưng model có thể học dần 
        #dần thông qua các epoch
            start = len(self._data) - self._batch_size
            end= len(self._data)
            self._num_epoch+=1 # chuyển sang epoch mới
            self._batch_id=0
        
            # xáo trộn data
            indices= list(range(len(self._data)))
            random.seed(2020)
            random.shuffle(indices)
            self._data, self._labels,self._sentence_lengths = self._data[indices], self._labels[indices], self._sentence_lengths[indices]

        return self._data[start:end], self._labels[start:end], self._sentence_lengths[start:end]


In [0]:
import tensorflow.compat.v1 as tf
import  numpy as np
MAX_DOC_LENGTH=500
NUM_CLASSES=20
tf.disable_eager_execution()

class RNN :
    def __init__(self, vocab_size, embedding_size, lstm_size, batch_size):
        self._vocab_size= vocab_size
        self._embedding_size= embedding_size
        self._lstm_size= lstm_size
        self._batch_size= batch_size

        self._data= tf.placeholder(tf.int32, shape=[batch_size,MAX_DOC_LENGTH])
        self._labels= tf.placeholder(tf.int32, shape=[batch_size,])
        self._sentence_lengths= tf.placeholder(tf.int32, shape=[batch_size,])
        # self._final_tokens= tf.placeholder(tf.int32, shape=[batch_size,])

    def embedding_layer(self, indices):
        pretrain_vectors= []
        pretrain_vectors.append(np.zeros(self._embedding_size))
        np.random.seed(2020)
        for i in range(self._vocab_size +1):
            pretrain_vectors.append(np.random.normal(
                loc=0.,
                scale=1.,
                size= self._embedding_size
            ))
        pretrain_vectors= np.array(pretrain_vectors)
        with tf.variable_scope("rnn_variables", reuse=tf.AUTO_REUSE) as scope:
            self._embedding_matrix= tf.get_variable(
                name='embedding',
                shape=(self._vocab_size +2,self._embedding_size ),
                initializer= tf.constant_initializer(pretrain_vectors)
            )
        return tf.nn.embedding_lookup(self._embedding_matrix, indices)

    def LSTM_layer(self, embeddings):
        lstm_cell= tf.nn.rnn_cell.BasicLSTMCell(self._lstm_size)
        zero_state= tf.zeros(shape=(self._batch_size, self._lstm_size))
        initial_state= tf.nn.rnn_cell.LSTMStateTuple(zero_state, zero_state)

        lstm_inputs= tf.unstack(
            tf.transpose(embeddings, perm=[1,0,2])
        )
        lstm_outputs, last_state= tf.nn.static_rnn(
            cell= lstm_cell,
            inputs= lstm_inputs,
            initial_state= initial_state,
            sequence_length= self._sentence_lengths
        ) #danh sách gồ 500 phần tử có dạng [num_doc, lstm_size]

        lstm_outputs= tf.unstack(
            tf.transpose(lstm_outputs, perm=[1,0,2])
        )
        lstm_outputs = tf.concat(lstm_outputs, axis=0)

        mask= tf.sequence_mask(
            lengths=self._sentence_lengths,
            maxlen=MAX_DOC_LENGTH,
            dtype=tf.float32
        )
        mask=tf.concat(tf.unstack(mask, axis=0), axis=0)
        mask= tf.expand_dims(mask, -1)

        lstm_outputs= mask*lstm_outputs
        lstm_outputs_splits=tf.split(lstm_outputs, num_or_size_splits=self._batch_size)
        lstm_outputs_sum=tf.reduce_sum(lstm_outputs_splits, axis=1)
        lstm_outputs_average= lstm_outputs_sum/ tf.expand_dims(
            tf.cast(self._sentence_lengths, tf.float32), -1
        )

        return lstm_outputs_average
    def build_graph(self):
        embeddings= self.embedding_layer(self._data)
        lstm_outputs= self.LSTM_layer(embeddings)

        with tf.variable_scope("rnn_variables", reuse= tf.AUTO_REUSE) as scope:
            weights= tf.get_variable(
                name="final_layer_weights",
                shape=(self._lstm_size, NUM_CLASSES),
                initializer=tf.random_normal_initializer(seed=2020)
            )
            biases = tf.get_variable(
                name="final_layer_biases",
                shape=(NUM_CLASSES),
                initializer=tf.random_normal_initializer(seed=2020)
            )
            logits= tf.matmul(lstm_outputs, weights) + biases

            label_one_hot= tf.one_hot(
                indices= self._labels,
                depth= NUM_CLASSES,
                dtype= tf.float32
            )

            loss= tf.nn.softmax_cross_entropy_with_logits(
                labels= label_one_hot,
                logits= logits
            )

            loss= tf.reduce_mean(loss)

            probs= tf.nn.softmax(logits)
            predicted_labels= tf.argmax(probs, axis=1)
            predicted_labels= tf.squeeze(predicted_labels)

        return  predicted_labels, loss


    def trainer(self,loss, learning_rate):
        with tf.variable_scope("rnn_variables", reuse=tf.AUTO_REUSE) as scope:
            train_op= tf.train.AdamOptimizer(learning_rate).minimize(loss)
        return train_op
    


In [3]:
def train_and_evaluate_RNN():
    with open('/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/vocab-raw.txt') as f:
        vocab_size= len(f.readlines())

    tf.set_random_seed(2020)
    rnn= RNN(
        vocab_size= vocab_size,
        embedding_size= 300,
        lstm_size= 50,
        batch_size=50
    )
    predicted_labels, loss= rnn.build_graph()
    train_op=rnn.trainer(loss=loss, learning_rate=0.01)

    with tf.Session() as sess:
        train_data_reader=DataReader(
            data_path='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-train-encoded.txt',
            batch_size=50,
            vocab_size= vocab_size
        )
        test_data_reader= DataReader(
            data_path='/content/drive/My Drive/Python code/DS_Lab2/Data/w2v/20news-test-encoded.txt',
            batch_size=50,
            vocab_size= vocab_size
        )
        step, MAX_STEP= 0,1500
        
        sess.run(tf.global_variables_initializer())
        while step < MAX_STEP:
            next_train_batch= train_data_reader.next_batch()
            # train_data, train_labels, train_sentence_lengths, train_final_tokens= next_train_batch
            train_data, train_labels, train_sentence_lengths= next_train_batch
            plabel_eval, loss_eval, _= sess.run(
                [predicted_labels, loss, train_op],
                feed_dict={
                    rnn._data : train_data,
                    rnn._labels: train_labels,
                    rnn._sentence_lengths: train_sentence_lengths,
                    # rnn._final_tokens: train_final_tokens            
                }
            )
            step+=1
            if step%20==0:
                print("step: "+ str(step)+" loss: "+ str(loss_eval))
            
            if train_data_reader._batch_id==0: #khi vua xet xong 1 epoch -> danh gia tren test data
                nums_true_pred=0
                while True:
                    next_test_batch= test_data_reader.next_batch()
                    # test_data, test_labels, test_sentence_lengths, test_final_tokens = next_test_batch
                    test_data, test_labels, test_sentence_lengths = next_test_batch
                    test_plabel_eval, loss_eval, _ = sess.run(
                        [predicted_labels, loss, train_op],
                        feed_dict={
                            rnn._data: test_data,
                            rnn._labels: test_labels,
                            rnn._sentence_lengths: test_sentence_lengths,
                            # rnn._final_tokens: test_final_tokens
                        }
                    )
                    matches= np.equal(test_plabel_eval, test_labels)
                    nums_true_pred += np.sum(matches.astype(float))
                    if test_data_reader._batch_id==0:
                        break
                print("Epoch: ",train_data_reader._num_epoch)
                print("Accuracy on test data: ", nums_true_pred * 100./len(test_data_reader._data))

if __name__=="__main__":
    with tf.variable_scope("rnn_variables", reuse= tf.AUTO_REUSE) as scope:
        train_and_evaluate_RNN()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step: 20 loss: 0.0002571087
step: 40 loss: 1.0598379
step: 60 loss: 7.5499415
step: 80 loss: 0.45277742
step: 100 loss: 1.4315
step: 120 loss: 5.4647174
step: 140 loss: 1.2392772
step: 160 loss: 4.03423
step: 180 loss: 4.7991905
step: 200 loss: 5.4172707
step: 220 loss: 4.50836
Epoch:  1
Accuracy o